# 環保署各測站的 PM2.5 資料

這個 jupyter notebook 提供了工具和範例，把環保署測站資料的 PM2.5 集合在一起，每個欄位是一個測站的時間序列。在使用之前，請先下載 2015 年環保署所有測站資料，並解壓縮放在 `../data/104_HOUR_00_20160323` 中。

首先，我們先載入需要用到的函式庫，以及在「資料清理」課程中使用到的一些函數：

In [1]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re, os

def detect_epa_nan(x):
    ''' Search for missing value symbol and assign np.nan '''
    if re.findall('\#|\*|x', str(x))!=[]:
        return(np.nan)
    else:
        return(x)

def detect_epa_norain(x):
    ''' Replace 'NR' (no-rain) with 0 '''
    if str(x)=='NR':
        return(0)
    else:
        return(x)

def clean_epa_station(x):
    ''' Clean up a EPA station dataset '''
    # Rename columns
    col_names = ['date','station','item','h00','h01','h02','h03','h04','h05','h06','h07','h08','h09',
                'h10','h11','h12','h13','h14','h15','h16','h17','h18','h19','h20','h21','h22','h23']
    x.columns = col_names
    # Process NA and NR
    floatdata = x.iloc[:,3:]
    floatdata = floatdata.applymap(detect_epa_nan)
    floatdata = floatdata.applymap(detect_epa_norain)
    floatdata.astype(np.float32)
    x.iloc[:,3:] = floatdata
    # Done
    return(x)

# Retrieve one item from EPA data and form a time series
def retrieve_epa_item(data, var):
    tmp = data.loc[data['item']==var,:]
    ts = pd.melt(tmp, id_vars=['date'], value_vars=tmp.keys()[3:], var_name='hour', value_name=var)
    ts[var] = ts[var].astype(np.float32)
    return(ts)


## 尋找目錄下的所有檔案

由於環保署資料包含了很多個檔案，又依照空品區的劃分，放在不同的資料夾底下，因此我們需要借助 python 的 [`os.walk()`](https://docs.python.org/3/library/os.html) 函數，幫我們自動「走遍」資料夾底下的每個檔案。

此外，由於資料的檔名包含了測站名稱，我們可以順便把測站名稱取出來，作為欄位的名稱，所以我們需要用到 [`str.find()`](https://www.tutorialspoint.com/python/string_find.htm) 函數，來幫我們找到測站名稱在檔名裡的位置。

In [2]:
# Walk through the specified path and find all files ended with 'xls'
def find_xls_files(path):
    ids = []    # 測站名稱
    urls = []   # 檔案完整路徑
    for root, dirs, files in os.walk(path):             # os.walk() 會傳回「根目錄」、「路徑」、和「檔名」三個字串 list
        for fname in files:                             # 每個檔名
            if(fname.endswith(".xls")):                 # 如果是以 xls 結尾
                fid = fname[(fname.find('104年')+4):][:(fname.find('_2016')-5)]
                ids.append(fid)
                urls.append(os.path.join(root, fname))
    return((ids,urls))


由於環保署資料的檔名是以 *YYY年XX站_YYYYMMDD.xls* 的格式命名 ，例如：104年花蓮站_20160320.xls，我們要取出字串裡的 *XX*，所以我們要：

1. 從完整檔名裡找到 *'104年'* 之後的字串 A
2. 從 A 裡找到 *'_2016'* 前面的字串，不包含「站」

因此，我們的寫法會是：

In [3]:
fn = '104年花蓮站_20160320.xls'
fn1 = fn[(fn.find('104年')+4):]
print(fn1)
fn2 = fn1[:(fn1.find('_2016')-1)]
print(fn2)

花蓮站_20160320.xls
花蓮


In [4]:
ids, epafiles = find_xls_files('../data/104_HOUR_00_20160323/')
print(ids)

[]


## 實際進行資料的讀取、清理與合併

接下來，我們就用上面的函數，讀進各個測站的資料，清理之後，抽出 PM2.5 的資料，然後用日期跟時間的欄位加以合併。

In [5]:
# Collect data from all files
f = epafiles[0]                                         # Start from the first file
tmp = pd.read_excel(f)                                  # Read in file
tmp = clean_epa_station(tmp)                            # Clean up nan and no-rain
data = retrieve_epa_item(tmp, 'PM2.5')                  # Retrieve PM2.5
nancounts = []
for f in epafiles[1:]:
    tmp = pd.read_excel(f)                                  # Read in file
    tmp = clean_epa_station(tmp)                            # Clean up nan and no-rain
    tmp = retrieve_epa_item(tmp, 'PM2.5')                   # Retrieve PM2.5
    data = data.merge(tmp, on=['date','hour'], how='left')  # Aggregate data
    nancounts.append(tmp.apply(lambda x: len(x)-x.count())) # Count the number of nan
    #print(f + ": " + str(len(tmp)))

data = data.sort_values(['date', 'hour'])                   # Sort data
cnames = list(data.columns[:2]) + ids
data.columns = cnames
data.head()

IndexError: list index out of range

## 資料的輸出

Python 的輸出/輸入非常多樣，我們這裡介紹 pandas.DataFrame 內建的輸出/輸入工具。在我們讀取資料的時候，使用了 [pandas.read_csv()](https://pandas.pydata.org/pandas-docs/stable/io.html#io-read-csv-table) 與 [pandas.read_excel()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html)，想當然耳，pandas 也提供了輸出成這兩種格式的工具 [pandas.DataFrame.to_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html) / [pandas.DataFrame.to_excel()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html)。

對於「表格式」的資料來說，CSV 大概是最常用的資料格式，下面我們就把整理好的資料輸出成 csv 檔，方便後續的分析來使用。

In [ ]:
data.to_csv('../data/pm25_2015.csv', index=False)